In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import html
import json
import time
from requests.exceptions import ConnectionError, TooManyRedirects
import unicodedata
import mysql.connector

In [2]:
con = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='news_db')

### Handling missing pubdate

In [8]:
raw = pd.read_sql("SELECT id, url, source FROM article WHERE pubdate = '1970-01-01 01:01:01'", con)
for index, row in raw.iterrows():
    print(row['url'])
    response = requests.get(row['url'])
    html = response.text
    soup = BeautifulSoup(html, "html5lib")
    pubdate = soup.find('meta', {'itemprop':'datePublished'})
    pubdate = pubdate['content'][:-2] if pubdate else '1970-01-01 01:01:01'
    pubdate = pubdate.strip(' \t\n\r')
    pubdate = datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%d %H:%M:%S"), '%Y-%m-%d %H:%M:%S')
    cursor = con.cursor()
    sql = "UPDATE article SET pubdate = '"+pubdate+"' WHERE id = "+str(row['id'])
    cursor.execute(sql)
    con.commit()
    cursor.close()